In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from config import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

In [2]:
# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [3]:
#Generate Cities List

# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

607

In [23]:
#perform weather check on each city using a series of successive API calls
#include a print log of each city as it's being processed with city number and city name

# Save config information.
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"

# Build partial query URL
query_url = f"{url}appid={api_key}&units={units}&q="

#url for GET requests
# url = "http://api.openweathermap.org/data/2.5/weather?q={city name}&appid={api_key}"

response = requests.get(url).json()

# Set empty lists to hold city weather data
max_temp = []
humidity = []
cloudiness = []
wind_speed = []
country = []
date = []
cities_list = []

# # Loop through each character
for city in cities[0: 10]:
    
#     # Create search query, make request and store in json
    query = query_url + city
    print(query)
    response = requests.get(query)
    response_json = response.json()
    print(response_json)
    
#     # Try to grab the weather data from API
    try:
        max_temp.append(response_json['main']['temp_max'])
        humidity.append(response_json['main']['humidity'])
        cloudiness.append(response_json['clouds']['all'])
        wind_speed.append(response_json['wind']['speed'])
        country.append(response_json['sys']['country'])
        date.append(response_json['dt'])
        cities_list.append(city)
        
        
        
#     # Handle exceptions for a character that is not available in the openweather API
    except:
        # Append null values
        print("City not found. Skipping...")
        pass
        

#perform weather check on each city using successive API calls with print log
        

http://api.openweathermap.org/data/2.5/weather?appid=2e8fc0916833705cacba448ba47295ea&units=metric&q=olinda
{'coord': {'lon': -34.86, 'lat': -8.01}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 26, 'feels_like': 27.61, 'temp_min': 26, 'temp_max': 26, 'pressure': 1012, 'humidity': 83}, 'visibility': 10000, 'wind': {'speed': 5.1, 'deg': 110}, 'clouds': {'all': 75}, 'dt': 1605034745, 'sys': {'type': 1, 'id': 8426, 'country': 'BR', 'sunrise': 1604994594, 'sunset': 1605039421}, 'timezone': -10800, 'id': 3393536, 'name': 'Olinda', 'cod': 200}
http://api.openweathermap.org/data/2.5/weather?appid=2e8fc0916833705cacba448ba47295ea&units=metric&q=geraldton
{'coord': {'lon': 114.6, 'lat': -28.77}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 10, 'feels_like': 7.94, 'temp_min': 10, 'temp_max': 10, 'pressure': 1017, 'humidity': 93}, 

In [24]:
print(max_temp)

[26, 10, 22.78, 19, 1.39, 27, 15, -1.3, 23.43, 9.44]
